In [3]:
import pandas as pd
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from multiprocessing import Pool, cpu_count
from matplotlib import patches, patheffects
import math
from pathlib import Path

from fastai.dataset import *
from fastai.conv_learner import *
from fastai.imports import *
from fastai.structured import *

import pdb

import pickle
from sklearn.model_selection import train_test_split

from IPython.display import FileLink
% load_ext autoreload
% autoreload 2
% matplotlib inline

In [4]:
ddir = "/home/edwin/Datasets/competitions/rsna-pneumonia-detection-challenge"
rsna_path = Path(ddir)

detailed_class_info_csv = pd.read_csv(rsna_path/'stage_1_detailed_class_info.csv')
stage_1_sample_submission_csv = pd.read_csv(rsna_path/'stage_1_sample_submission.csv')
stage_1_train_labels_csv = pd.read_csv(rsna_path/'stage_1_train_labels.csv')

In [5]:
BBOX_CSV = '../tmp/rsna_bbox.csv'
TRN = 'train'
TEST = 'test2'

In [6]:
f_model=resnet34
sz=224
bs=64

tfms = tfms_from_model(f_model, sz, crop_type=CropType.NO)

In [35]:
md = ImageClassifierData.from_csv(rsna_path, TRN, BBOX_CSV, tfms=tfms, bs=64, test_name=TEST, continuous=True, suffix='.dcm')

In [1]:
# n_sample

In [12]:
TRN_FILES = list((rsna_path/TRN).glob('*.dcm'))

In [24]:
RANDOM_TRN_FILES = np.random.shuffle(TRN_FILES)

In [31]:
n_sample = int(0.10 * len(TRN_FILES))

In [21]:
(rsna_path/'sample').mkdir(exist_ok=True)
(rsna_path/'sample/train').mkdir(exist_ok=True)
(rsna_path/'sample/test').mkdir(exist_ok=True)

In [23]:
! ls {rsna_path/'sample'}

test  train


In [28]:
len(stage_1_train_labels_csv)

28989

In [39]:
df_sample = stage_1_train_labels_csv.sample(n=n_sample).reset_index(); len(df_sample)

2568

In [41]:
SAMPLE_TRN_LABELS = '../tmp/rsna_train_sample.feather'

In [40]:
df_sample.to_feather(SAMPLE_TRN_LABELS)

In [42]:
df_sample = pd.read_feather(SAMPLE_TRN_LABELS)

In [73]:
def bbox_corners(row):
    x = row['x']
    y = row['y']
    width = row['width']
    height = row['height'] 
    if math.isnan(x):
        return "0 0 0 0"
    
    x2 = x + width
    y2 = y + height
    return f"{x} {y} {x2} {y2}" 

In [50]:
# of rows where Target = 1
n_rows_target_1 = len(df_sample[df_sample['Target'] != 0])
n_rows_target_1 / n_sample

0.3072429906542056

In [75]:
df_sample['bbox_corners'] = df_sample.apply(bbox_corners, axis=1)

In [79]:
df_sample_bbox_corners = df_sample[['patientId', 'bbox_corners']]

In [82]:
BBOX_CSV_CORNERS = '../tmp/df_sample_bbox_corners.csv'

In [81]:
df_sample_bbox_corners.to_csv(BBOX_CSV_CORNERS, index=False)

In [83]:
df_sample_bbox_corners = pd.read_csv(BBOX_CSV_CORNERS)

In [84]:
df_sample_bbox_corners[:10]

,patientId,bbox_corners
0,cdff09a5-ca16-4391-863b-ae2cc6946018,0 0 0 0
1,7e25dcde-dc45-41ca-85f6-a73480915d93,0 0 0 0
2,3fd032bc-c98f-4fc9-8a4b-903797721666,0 0 0 0
3,0beac0c9-331a-4aad-9cec-099aff98c399,614.0 176.0 777.0 356.0
4,ffa424d2-6e6b-4eed-93ab-7551e8941215,187.0 462.0 367.0 681.0
5,a4a3fd3c-331e-43d8-8e21-1d9162d34424,0 0 0 0
6,9f29797b-280d-4b7b-9c7a-aba284959b93,0 0 0 0
7,614af470-e19d-4013-baa2-eae7e2b0198a,0 0 0 0
8,98e42965-bd5b-44b2-92de-38f8aa015341,0 0 0 0
9,bfbd4ed3-1d78-437b-b5dc-09ab063b55fa,242.0 252.0 437.0 620.0
